# Load data

In [5]:
fp = open("/Users/emma/Desktop/uid_score_bid.dat.csv","r")

In [6]:
users = {}

In [9]:
for line in open("/Users/emma/Desktop/uid_score_bid.dat.csv"):
    lines = line.strip().split(",")
    if lines[0] not in users:
        users[lines[0]] = {}
    users[lines[0]][lines[2]]=float(lines[1])

In [13]:
from math import sqrt  

# Build Recommendation system

k:the number of neighbors

metric: methods to caculate similaries

n: number of recommendation books

# user based CF

In [16]:
class recommender:
    def __init__(self, data, k=3, metric='pearson', n=12):
        self.k = k
        self.n = n
        self.username2id = {}
        self.userid2name = {}
        self.productid2name = {}
        
        self.metric = metric
        if self.metric == 'pearson':
            self.fn = self.pearson
        if type(data).__name__ == 'dict':
            self.data = data
        
    def convertProductID2name(self, id):
        if id in self.productid2name:
            return self.productid2name[id]
        else:
            return id
        
    # use pearson to caculate similaries:
    def pearson(self, rating1, rating2):
        sum_xy = 0
        sum_x =0
        sum_y = 0
        sum_x2 = 0
        sum_y2 = 0
        n = 0
        for key in rating1:
            if key in rating2:
                n += 1
                x = rating1[key]
                y = rating2[key]
                sum_xy += x * y
                sum_x += x
                sum_y += y
                sum_x2 += pow(x, 2)
                sum_y2 += pow(y, 2)
        if n == 0:
            return 0
        
        denominator = sqrt(sum_x2 - pow(sum_x, 2) / n) * sqrt(sum_y2 - pow(sum_y, 2) / n)
        if denominator == 0:
            return 0
        else:
            return (sum_xy - (sum_x * sum_y) / n) / denominator
        
    def computeNearestNeighbor(self, username):
        distances = []
        for instance in self.data:
            if instance != username:
                distance = self.fn(self.data[username],self.data[instance])
                distances.append((instance, distance))
                
        distances.sort(key=lambda artistTuple: artistTuple[1], reverse=True)
        return distances
    
    #recommendation algorithm
    def recommend(self, user):
        recommendations = {}
        nearest = self.computeNearestNeighbor(user)
        
        userRatings = self.data[user]
        totalDistance = 0.0
        
        for i in range(self.k):
            totalDistance += nearest[i][1]
        if totalDistance == 0.0:
            totalDistance=1.0
            
        for i in range(self.k):
            weight = nearest[i][1] / totalDistance
            
            name = nearest[i][0]
            
            neighborRatings = self.data[name]
            
            for artist in neighborRatings:
                if not artist in userRatings:  
                    if artist not in recommendations:  
                        recommendations[artist] = (neighborRatings[artist] * weight)
                    else: 
                        recommendations[artist] = (recommendations[artist]+ neighborRatings[artist] * weight)
                        
        recommendations = list(recommendations.items())  
        recommendations = [(self.convertProductID2name(k), v)for (k, v) in recommendations] 
        
        recommendations.sort(key=lambda artistTuple: artistTuple[1], reverse = True)  
  
        return recommendations[:self.n],nearest

def adjustrecommend(id):
    bookid_list = []
    r = recommender(users)
    k,nearuser = r.recommend("%s" % id)
    for i in range(len(k)):
        bookid_list.append(k[i][0])
    return bookid_list,nearuser[:15]           

In [17]:
bookid_list,near_list = adjustrecommend("changanamei")  
print ("bookid_list:",bookid_list)  
print ("near_list:",near_list) 

bookid_list: ['2143732', '4105446', '3315879', '1119522', '1059419', '1073744', '1431870', '1090043', '4742918', '10594787', '1057244', '2154960']
near_list: [('122946019', 0.9547859244962533), ('56746289', 0.8280786712108233), ('yiminuansheng', 0.8236877675803737), ('4030281', 0.8209008497548285), ('46832091', 0.816666666666671), ('111223566', 0.812420748459488), ('4750931', 0.8017837257372784), ('65121529', 0.8006407690254372), ('57475649', 0.7883561227922886), ('budingsetlla', 0.7784989441615238), ('69874649', 0.7698003589195094), ('122191787', 0.7654779007031404), ('bshayna', 0.7500000000000061), ('94986189', 0.7500000000000061), ('53728385', 0.7500000000000061)]


# item based CF

In [18]:
import math

In [23]:
class ItemBasedCF:  
    def __init__(self,train_file):  
        self.train_file = train_file  
        self.readData()  
    def readData(self):  
        self.train = dict()     #user-item rating and traindataset  
        for line in open(self.train_file):  
            # user,item,score = line.strip().split(",")  
            user,score,item = line.strip().split(",")  
            self.train.setdefault(user,{})  
            self.train[user][item] = int(float(score)) 
            
    def ItemSimilarity(self):  
        #build item metrics  
        C = dict()  #items  
        N = dict()  #items be purchased by different users  
        for user,items in self.train.items():  
            for i in items.keys():  
                N.setdefault(i,0)  
                N[i] += 1  
                C.setdefault(i,{})  
                for j in items.keys():  
                    if i == j : continue  
                    C[i].setdefault(j,0)  
                    C[i][j] += 1  
                    
    #caculate the simalities of item metrics
        self.W = dict()  
        for i,related_items in C.items():  
            self.W.setdefault(i,{})  
            for j,cij in related_items.items():  
                self.W[i][j] = cij / (math.sqrt(N[i] * N[j]))  
        return self.W 
    
    #top k relevant users
    def Recommend(self,user,K=3,N=10):  
        rank = dict()  
        action_item = self.train[user]     #user rating on the items  
        for item,score in action_item.items():  
            for j,wj in sorted(self.W[item].items(),key=lambda x:x[1],reverse=True)[0:K]:  
                if j in action_item.keys():  
                    continue  
                rank.setdefault(j,0)  
                rank[j] += score * wj  
        return dict(sorted(rank.items(),key=lambda x:x[1],reverse=True)[0:N]) 

In [28]:
Item = ItemBasedCF("/Users/emma/Desktop/uid_score_bid.dat.csv")  
Item.ItemSimilarity()  
recommedDic = Item.Recommend("xiyuweilan")

{'1040104': 7.92927319994814,
 '1080309': 8.181614384131635,
 '1082138': 7.724094115315923,
 '1085799': 8.203142976157213,
 '1119522': 11.81651002922847,
 '1254588': 12.833112463864879,
 '26305561': 6.248165542160276,
 '26384985': 6.36064881658183,
 '26669243': 6.969721285190005,
 '3131626': 11.34269062173854}

In [29]:
for k,v in recommedDic.items():  
    print(k,"\t",v)

1254588 	 12.833112463864879
1119522 	 11.81651002922847
3131626 	 11.34269062173854
1085799 	 8.203142976157213
1080309 	 8.181614384131635
1040104 	 7.92927319994814
1082138 	 7.724094115315923
26669243 	 6.969721285190005
26384985 	 6.36064881658183
26305561 	 6.248165542160276
